# Cifar10 - exercicio de classificar 3 classes

Fazer classificação de 3 classes usando rede neural convolucional.
Não utilizar o pacote sklearn. Apenas o Keras e o NumPy.

Não utilizar a função de acompanhamento de gráficos durante o treinamento.

Gerar uma figura mosaic que contenha as 5 imagens de classificação correta de menor probabilidade de predição.

Gerar esta figura com o nome: cifar_fig.png

## Importação de bibliotecas

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential, load_model, Model, Input
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import keras.regularizers as reg
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_floatx('float32')

## Carregamento do dataset

In [3]:
data = np.load('/etc/jupyterhub/ia368z_2s2017/datasets/cifar10-redux.npz')

In [4]:
X = data['X_train']
y = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

X = np.swapaxes(X, 1, 2)
X = np.swapaxes(X, 2, 3)
X_test = np.swapaxes(X_test, 1, 2)
X_test = np.swapaxes(X_test, 2, 3)

In [5]:
X.shape, y.shape, X_test.shape, y_test.shape

((2000, 32, 32, 3), (2000,), (500, 32, 32, 3), (500,))

In [6]:
X.dtype, y.dtype, X_test.dtype, y_test.dtype

(dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('int64'))

## Ajuste dos dados

In [7]:
X = X.astype('float32')
X /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

# transforma labels em 0, 1 e 2, ao invés de 3, 4 e 5
y = y - 3
y_test = y_test - 3

## Separação dos dados de treinamento em treinamento e validação

In [8]:
# numero de amostras
nb_data = X.shape[0]

# semente fixa para dar reproducibilidade
seed = 13
np.random.seed(seed)

msk = np.random.rand(nb_data) < 0.80
X_train = X[msk]
X_validate = X[~msk]
y_train = y[msk]
y_validate = y[~msk]

nb_train = X_train.shape[0]
nb_validate = X_validate.shape[0]

print(nb_train)
print(nb_validate)
print(X_train.shape)
print(X_validate.shape)
print(y_train.shape)
print(y_validate.shape)

1605
395
(1605, 32, 32, 3)
(395, 32, 32, 3)
(1605,)
(395,)


## Transforma o vetor de labels para o formato de one-hot encoding.

In [9]:
nb_classes = 3

y_oh = np_utils.to_categorical(y, nb_classes)
y_train_oh = np_utils.to_categorical(y_train, nb_classes)
y_validate_oh = np_utils.to_categorical(y_validate, nb_classes)
y_test_oh = np_utils.to_categorical(y_test, nb_classes)

## Criando a rede VGG16

In [29]:
i = Input(shape=(32,32, 3))
vgg = VGG16(include_top=False, weights='imagenet', input_tensor=i)

In [30]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

## Ajustando a rede para os dados do CIFAR-10

In [31]:
# com a rede completa: #41%
#vgg.layers.pop() # 48.80%
#vgg.layers.pop() # 45.00%
#vgg.layers.pop() # 44.80%
#vgg.layers.pop() # 46.80%

vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

## Obtendo as features com a VGG

In [32]:
train_features = vgg.predict(X_train)
valid_features = vgg.predict(X_validate)
test_features = vgg.predict(X_test)

print(train_features.shape[1:])
print(valid_features.shape)
print(test_features.shape)

(1, 1, 512)
(395, 1, 1, 512)
(500, 1, 1, 512)


## Construíndo a CNN com o Keras

In [33]:
# Definindo a rede
model = Sequential()
model.add(Flatten(input_shape=train_features.shape[1:]))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.75))
model.add(Dense(nb_classes, kernel_regularizer=reg.l2(0.025)))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_4 (Activation)    (None, 3)                 0         
Total params: 66,051
Trainable params: 66,051
Non-trainable params: 0
_________________________________________________________________


## Compilando a rede

In [34]:
loss = 'categorical_crossentropy'
#opt = RMSprop()
opt = Adam()

model.compile(loss=loss, optimizer=opt, metrics=["accuracy"]) 

## Callbacks

In [35]:
checkpointer = ModelCheckpoint(filepath='cifar10.hdf5', monitor='val_acc', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
lrreduction = ReduceLROnPlateau(monitor='val_acc', patience=7, verbose=1, factor=0.8, min_lr=0.00001)

## Treinamento da rede

In [36]:
batch_size = 40
epochs = 100

model.fit(train_features, y_train_oh, 
          batch_size=batch_size, 
          epochs=epochs,
          validation_data=(valid_features, y_validate_oh),
          callbacks=[checkpointer, lrreduction])
          #callbacks=[checkpointer, earlystopper, lrreduction])

Train on 1605 samples, validate on 395 samples
Epoch 1/100
1605/1605 [==============================] - 0s - loss: 1.2851 - acc: 0.4436 - val_loss: 1.0115 - val_acc: 0.6177
Epoch 2/100
1605/1605 [==============================] - 0s - loss: 1.0341 - acc: 0.5483 - val_loss: 0.9415 - val_acc: 0.6177
Epoch 3/100
1605/1605 [==============================] - 0s - loss: 0.9606 - acc: 0.5894 - val_loss: 0.8920 - val_acc: 0.6278
Epoch 4/100
1605/1605 [==============================] - 0s - loss: 0.9051 - acc: 0.6187 - val_loss: 0.8667 - val_acc: 0.6354
Epoch 5/100
1605/1605 [==============================] - 0s - loss: 0.8850 - acc: 0.6343 - val_loss: 0.8480 - val_acc: 0.6456
Epoch 6/100
1605/1605 [==============================] - 0s - loss: 0.8509 - acc: 0.6617 - val_loss: 0.8258 - val_acc: 0.6506
Epoch 7/100
1605/1605 [==============================] - 0s - loss: 0.8365 - acc: 0.6517 - val_loss: 0.8262 - val_acc: 0.6532
Epoch 8/100
1605/1605 [==============================] - 0s - loss: 0.8

1605/1605 [==============================] - 0s - loss: 0.5786 - acc: 0.7863 - val_loss: 0.7731 - val_acc: 0.6506
Epoch 65/100
1605/1605 [==============================] - 0s - loss: 0.5767 - acc: 0.7838 - val_loss: 0.7758 - val_acc: 0.6658
Epoch 66/100
1605/1605 [==============================] - 0s - loss: 0.5717 - acc: 0.7900 - val_loss: 0.7757 - val_acc: 0.6582
Epoch 67/100
1605/1605 [==============================] - 0s - loss: 0.5799 - acc: 0.7844 - val_loss: 0.7752 - val_acc: 0.6582
Epoch 68/100
1480/1605 [==========================>...] - ETA: 0s - loss: 0.5702 - acc: 0.7993Epoch 00067: val_acc did not improve

Epoch 00067: reducing learning rate to 0.0002621440216898918.
1605/1605 [==============================] - 0s - loss: 0.5696 - acc: 0.8000 - val_loss: 0.7765 - val_acc: 0.6633
Epoch 69/100
1605/1605 [==============================] - 0s - loss: 0.5620 - acc: 0.7981 - val_loss: 0.7751 - val_acc: 0.6684
Epoch 70/100
1605/1605 [==============================] - 0s - loss: 0

1605/1605 [==============================] - 0s - loss: 0.5528 - acc: 0.7956 - val_loss: 0.7841 - val_acc: 0.6532
Epoch 97/100
1605/1605 [==============================] - 0s - loss: 0.5501 - acc: 0.8081 - val_loss: 0.7863 - val_acc: 0.6582
Epoch 98/100
1605/1605 [==============================] - 0s - loss: 0.5410 - acc: 0.8212 - val_loss: 0.7865 - val_acc: 0.6557
Epoch 99/100
1605/1605 [==============================] - 0s - loss: 0.5440 - acc: 0.8118 - val_loss: 0.7855 - val_acc: 0.6557
Epoch 100/100
1605/1605 [==============================] - 0s - loss: 0.5399 - acc: 0.8106 - val_loss: 0.7860 - val_acc: 0.6608


## Apaga modelo atual

In [30]:
del model

## Carrega modelo salvo em disco

In [37]:
model = load_model('cifar10.hdf5')

## Avaliação do treinamento no conjunto de testes com o melhor modelo

In [38]:
loss, accuracy = model.evaluate(test_features, y_test_oh, batch_size=10)
print("\n[INFO] accuracy on the test data set: {:.2f}% [{:.5f}]".format(accuracy * 100, loss))

250/500 [==============>...............] - ETA: 0s
[INFO] accuracy on the test data set: 68.60% [0.75198]


## Construindo a nova rede com Fine Tunning

In [10]:
vgg = VGG16(include_top=False, weights='imagenet', classes=nb_classes, pooling='max')

In [11]:
#vgg.output.set_shape([None, 1, 1, 512])
#x = Flatten(input_shape=vgg.output.shape[1:])(vgg.output)
x = Dense(1728, activation='relu', name='f1')(vgg.output)
x = Dense(128, activation='relu', name='f2')(x)
x = Dropout(0.75)(x)
x = Dense(nb_classes, activation='softmax', kernel_regularizer=reg.l2(0.025), name='f3')(x)

model = Model(inputs=vgg.input, outputs=x)

model.summary()

w0, b0, w1, b1, w2, b2, w3, b3, w4, b4 = load_model('my_cifar_dataplus.hdf5').get_weights()

# Coloco nas camadas densas finais da rede
model.layers[21].set_weights([w3, b3])
model.layers[23].set_weights([w4, b4])

# camadas que não serão treinadas
for layer in model.layers[:15]:
        layer.trainable = False

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Compilando a rede

In [41]:
loss = 'categorical_crossentropy'
#opt = RMSprop()
opt = Adam()
#opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)

model.compile(loss=loss, optimizer=opt, metrics=["accuracy"]) 

## Callbacks

In [42]:
checkpointer = ModelCheckpoint(filepath='cifar10.hdf5', monitor='val_acc', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
lrreduction = ReduceLROnPlateau(monitor='val_acc', patience=7, verbose=1, factor=0.8, min_lr=0.00001)

## Fine tunning da rede

In [43]:
batch_size = 10
epochs = 50

model.fit(X_train, y_train_oh, 
          batch_size=batch_size, 
          epochs=epochs,
          validation_data=(X_validate, y_validate_oh),
          callbacks=[checkpointer, earlystopper, lrreduction])

Train on 1605 samples, validate on 395 samples
Epoch 1/50
1605/1605 [==============================] - 7s - loss: 1.2490 - acc: 0.3651 - val_loss: 1.2251 - val_acc: 0.3241
Epoch 2/50
1605/1605 [==============================] - 5s - loss: 1.1639 - acc: 0.3396 - val_loss: 1.1408 - val_acc: 0.3241
Epoch 3/50
1605/1605 [==============================] - 5s - loss: 1.1332 - acc: 0.3514 - val_loss: 1.1286 - val_acc: 0.3241
Epoch 4/50
1605/1605 [==============================] - 5s - loss: 1.1224 - acc: 0.3514 - val_loss: 1.1203 - val_acc: 0.3241
Epoch 5/50
1605/1605 [==============================] - 5s - loss: 1.1152 - acc: 0.3514 - val_loss: 1.1148 - val_acc: 0.3241
Epoch 6/50
1605/1605 [==============================] - 5s - loss: 1.1100 - acc: 0.3514 - val_loss: 1.1105 - val_acc: 0.3241
Epoch 7/50
1605/1605 [==============================] - 5s - loss: 1.1064 - acc: 0.3514 - val_loss: 1.1078 - val_acc: 0.3241
Epoch 8/50
1605/1605 [==============================] - 5s - loss: 1.1040 - ac

## Apaga modelo atual

In [44]:
del model

## Carrega modelo salvo em disco

In [45]:
model = load_model('cifar10.hdf5')

## Avaliação do treinamento no conjunto de testes com o melhor modelo

In [46]:
loss, accuracy = model.evaluate(X_test, y_test_oh, batch_size=10)
print("\n[INFO] accuracy on the test data set: {:.2f}% [{:.5f}]".format(accuracy * 100, loss))

420/500 [========================>.....] - ETA: 0s
[INFO] accuracy on the test data set: 34.60% [1.20269]
